Elliot Ide Pozo

- <del>Crear script en python que permita extraer la data desde el server de base de datos (activismo.inf.uach.cl)</del>
- <del>Crear script en python que permita crear un json (dataset) de desafío, usuario, consulta, resultado (diccionario).</del>
- <del>Crear un script para obtener la representación tf-idf (NLTK,,,).</del>
- <del>Crear un script que calcule al menos una métrica de las establecidas por cada conjunto.</del>


In [1]:
from sshtunnel import SSHTunnelForwarder
import pymysql
import pandas as pd

In [2]:
# https://stackoverflow.com/questions/21903411/enable-python-to-connect-to-mysql-via-ssh-tunnelling
# https://stackoverflow.com/questions/42726681/conecting-to-mysql-in-a-remote-server-from-python

with SSHTunnelForwarder(
            'activismo.inf.uach.cl',
            ssh_username='vps238',
            ssh_password='vps238.2018',
            remote_bind_address=('127.0.0.1', 3306)) as tunnel:
        conn = pymysql.connect(host='127.0.0.1', user='learning',
                               passwd='analytic$',db='gonsa2',
                               port=tunnel.local_bind_port)
        query = "SELECT date_executed, FK_challenge_id_number, FK_student_nick,id_number,query,snippet \
        FROM Students_query\
        INNER JOIN Students_queries_results ON FK_student_query_id_number = id_number\
        WHERE FK_challenge_id_number = 1 AND FK_student_nick != 'test_student'\
        LIMIT 20";
        data = pd.read_sql(query, conn)
        conn.close()

In [3]:
from bs4 import BeautifulSoup

In [4]:
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/
#https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas
    
# transformar en un objeto BeautifulSoup, para luego retornar el texto sin tags
data['snippet'] = data['snippet'].map(lambda text: BeautifulSoup(text, 'html.parser').get_text())

In [5]:
data

,date_executed,FK_challenge_id_number,FK_student_nick,id_number,query,snippet
0,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,vih en chile. Es urgente aumentar el diagnósti...
1,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,"En Chile, la Ley 19.779 resguarda los derechos..."
2,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,¿Cuánto tarda una persona con VIH en llegar a ...
3,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,VIH Sida en Chile: Cifras del Minsal indican q...
4,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,En los últimos diez años los contagios de VIH ...
5,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Advierten que el VIH sida está descontrolado e...
6,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,SIDA en Chile: la epidemia que crece silencios...
7,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,del tratamiento anti-vih para ayudar y protege...
8,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Para el presidente de la Corporación Sida Chil...
9,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Informe revela que Chile es el país donde más ...


### Estimación TF-IDF 

Fuente: 

https://www.youtube.com/watch?v=hc3DCn8viWs
    
**Drawbacks**
- Only based on terms
- Weak on capturing document topic
- Weak handling synonyms

**How to overcome drawbacks**
- LSA (Latent Semantic Analysis)
- Word Embedding (Word2Vec, Glove)
- ConceptNet

In [6]:
q = pd.Series(data['query'][0])

In [7]:
corpus = q.append(data['snippet'][:6], ignore_index=True) # q, d1, d2 y d3
corpus

0                                         vih en chile
1    vih en chile. Es urgente aumentar el diagnósti...
2    En Chile, la Ley 19.779 resguarda los derechos...
3    ¿Cuánto tarda una persona con VIH en llegar a ...
4    VIH Sida en Chile: Cifras del Minsal indican q...
5    En los últimos diez años los contagios de VIH ...
6    Advierten que el VIH sida está descontrolado e...
dtype: object

In [8]:
import nltk
from nltk.corpus import stopwords

es_sw = stopwords.words('spanish') # lista stopword a usar

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tf = TfidfVectorizer(stop_words=es_sw) # crear la transformación

tfidf_matrix = tf.fit_transform(corpus)

In [11]:
# https://stackoverflow.com/questions/34449127/sklearn-tfidf-transformer-how-to-get-tf-idf-values-of-given-words-in-documen

feature_names = tf.get_feature_names()
print(feature_names, len(feature_names))

['15', '19', '2016', '2018', '25', '67', '779', '96', 'acceso', 'adecuadamente', 'administración', 'adolescente', 'advierten', 'afectados', 'anterior', 'asimismo', 'aumentado', 'aumentar', 'años', 'brote', 'calcula', 'campañas', 'capacidad', 'cartera', 'casos', 'cautela', 'chile', 'ciento', 'cifras', 'contagia', 'contagios', 'control', 'corporación', 'crecieron', 'cuánto', 'cómo', 'derechos', 'desarrollar', 'descontrolado', 'diagnosticada', 'diagnóstico', 'diez', 'dificultades', 'discriminaciones', 'edad', 'educación', 'ejercicio', 'epidémico', 'estudiada', 'etapa', 'falta', 'grupo', 'igualitario', 'impidiendo', 'indican', 'infección', 'infectada', 'infectadas', 'infectarse', 'joven', 'latinoamérica', 'lejos', 'ley', 'libre', 'lidera', 'llegar', 'mejorar', 'ministra', 'minsal', 'mitad', 'mundial', 'opción', 'organización', 'periodo', 'persona', 'personas', 'población', 'prevalencia', 'prevención', 'principales', 'producen', 'puedo', 'realizarse', 'resguarda', 'respecto', 'respuesta', '

In [12]:
#https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76

dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,15,19,2016,2018,25,67,779,96,acceso,adecuadamente,...,tarda,test,trabaja,trabajo,tratamiento,urgente,ventana,vih,viven,últimos
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.216217,...,0.000000,0.000000,0.000000,0.000000,0.179479,0.216217,0.000000,0.090608,0.000000,0.000000
2,0.000000,0.185223,0.000000,0.000000,0.000000,0.000000,0.185223,0.000000,0.185223,0.000000,...,0.000000,0.000000,0.000000,0.185223,0.153751,0.000000,0.000000,0.155239,0.185223,0.000000
3,0.000000,0.000000,0.163046,0.163046,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.163046,0.000000,0.000000,0.000000,0.000000,0.000000,0.163046,0.204978,0.000000,0.000000
4,0.228361,0.000000,0.000000,0.000000,0.228361,0.000000,0.000000,0.228361,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095697,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.211202,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.211202,0.000000,0.000000,0.000000,0.000000,0.000000,0.088506,0.000000,0.211202
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.283755,0.000000,0.000000,0.000000,0.000000,0.118910,0.000000,0.000000


In [13]:
print(tfidf_matrix[1])

  (0, 100)	0.1794789884674251
  (0, 31)	0.1794789884674251
  (0, 39)	0.2162173733390351
  (0, 56)	0.1794789884674251
  (0, 69)	0.2162173733390351
  (0, 20)	0.2162173733390351
  (0, 48)	0.2162173733390351
  (0, 9)	0.2162173733390351
  (0, 91)	0.4324347466780702
  (0, 60)	0.2162173733390351
  (0, 55)	0.2162173733390351
  (0, 77)	0.2162173733390351
  (0, 57)	0.2162173733390351
  (0, 75)	0.3589579769348502
  (0, 40)	0.1794789884674251
  (0, 17)	0.2162173733390351
  (0, 101)	0.2162173733390351
  (0, 26)	0.18121601162193923
  (0, 103)	0.09060800581096962


### Similaridad del coseno

Fuentes:

https://intellipaat.com/community/1103/python-tf-idf-cosine-to-find-document-similarity

In [14]:
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [15]:
S = []
for i in range(corpus.count()):
    S.append(linear_kernel(tfidf_matrix[i], tfidf_matrix).flatten()) # equivalente a cos

S = np.asarray(S)
S

array([[1.        , 0.19220861, 0.16465622, 0.1932551 , 0.13533588,
        0.1251668 , 0.16816463],
       [0.19220861, 1.        , 0.22129782, 0.05524548, 0.02601272,
        0.02405814, 0.03232269],
       [0.16465622, 0.22129782, 1.        , 0.11312296, 0.03833506,
        0.04756454, 0.08384885],
       [0.1932551 , 0.05524548, 0.11312296, 1.        , 0.06854225,
        0.02418912, 0.08516872],
       [0.13533588, 0.02601272, 0.03833506, 0.06854225, 1.        ,
        0.11663779, 0.04734847],
       [0.1251668 , 0.02405814, 0.04756454, 0.02418912, 0.11663779,
        1.        , 0.02104863],
       [0.16816463, 0.03232269, 0.08384885, 0.08516872, 0.04734847,
        0.02104863, 1.        ]])

### Estimacion MMR

Fuentes:

http://www.cs.bilkent.edu.tr/~canf/CS533/hwSpring14/eightMinPresentations/handoutMMR.pdf
https://www.quora.com/What-is-maximal-marginal-relevance-MMR

MMR is a measure for quantifying the extend of dissimilarity between the item being considered and those already selected.
Higher MMR means the considered item is both relevant to the query and contains minimal similarity to previous selected items.

In [16]:
S

array([[1.        , 0.19220861, 0.16465622, 0.1932551 , 0.13533588,
        0.1251668 , 0.16816463],
       [0.19220861, 1.        , 0.22129782, 0.05524548, 0.02601272,
        0.02405814, 0.03232269],
       [0.16465622, 0.22129782, 1.        , 0.11312296, 0.03833506,
        0.04756454, 0.08384885],
       [0.1932551 , 0.05524548, 0.11312296, 1.        , 0.06854225,
        0.02418912, 0.08516872],
       [0.13533588, 0.02601272, 0.03833506, 0.06854225, 1.        ,
        0.11663779, 0.04734847],
       [0.1251668 , 0.02405814, 0.04756454, 0.02418912, 0.11663779,
        1.        , 0.02104863],
       [0.16816463, 0.03232269, 0.08384885, 0.08516872, 0.04734847,
        0.02104863, 1.        ]])

In [17]:
from collections import OrderedDict, Counter

In [18]:
docs = list(range(1,corpus.count()))

In [19]:
lambda_ = 0.5

In [20]:
#https://pymotw.com/2/collections/ordereddict.html

selected = OrderedDict()
init_idx= np.argmax(S[0][1::]) + 1 # el indice del d con mayor similaridad a q
selected[init_idx] = S[init_idx,0]

In [21]:
docs, selected

([1, 2, 3, 4, 5, 6], OrderedDict([(3, 0.19325509863324558)]))

In [22]:
#https://www.quora.com/Where-can-I-find-a-maximum-marginal-relevance-algorithm-in-Python-for-redundancy-removal-in-two-documents

while Counter(selected.keys()) != Counter(docs): # comparar la lista de docs con los seleccionados
    remaining = [x for x in docs if x not in selected]
    mmr_score = lambda x: lambda_*S[x, 0] - (1-lambda_)*max([S[x, y] for y in selected])
    result = {k: mmr_score(k) for k in remaining}
    next_selected = max(result, key=result.get) 
    selected[next_selected] = result[next_selected]

In [23]:
selected

OrderedDict([(3, 0.19325509863324558),
             (1, 0.06848156321586256),
             (5, 0.050488840244009875),
             (6, 0.04149795545339412),
             (4, 0.009349044287457836),
             (2, -0.028320796616313815)])